## Credit Card Consumption Prediction 信用卡消費預測
- 以此三個月的資料預測未來三個月的信用卡消費。

### 預測
- 信用卡在未來三個月內的平均花費 (cc_cons)

### 特徵定義
- id &rArr; 每個客戶的唯一ID
- account_type &rArr; 帳戶類型 - [活期帳戶&rArr;Current accounts] 或 [儲蓄帳戶：Saving accounts]
- gender &rArr; 客戶的性別
- age &rArr; 客戶的年齡
- region_code &rArr; 要求居住地區（有訂單）
- cc_cons_apr &rArr; 信用卡credit card 在 4月份花費
- dc_cons_apr &rArr; 扣賬卡Debit Card 在 4月份花費
- cc_cons_may &rArr; 信用卡credit card 在 5月份花費
- dc_cons_may &rArr; 扣賬卡Debit Card 在 5月份花費
- cc_cons_jun &rArr; 信用卡credit card 在 6月份花費
- dc_cons_jun &rArr; 扣賬卡Debit Card 在 6月份花費
- cc_count_apr &rArr; 信用卡credit card 在 4月份交易數量
- cc_count_may &rArr; 信用卡credit card 在 5月份交易數量
- cc_count_jun &rArr; 信用卡credit card 在 6月份交易數量
- dc_count_apr &rArr; 扣賬卡Debit Card 在 4月份交易數量
- dc_count_may &rArr; 扣賬卡Debit Card 在 5月份交易數量
- dc_count_jun &rArr; 扣賬卡Debit Card 在 6月份交易數量
- card_lim &rArr; 信用卡最大限制金額
- personal_loan_active &rArr; 與其他銀行有效的個人貸款
- vehicle_loan_active &rArr; 與其他銀行有效的車輛貸款
- personal_loan_closed &rArr; 在過去12個月內關閉個人貸款
- vehicle_loan_closed &rArr; 過去12個月的關閉車輛貸款
- investment_1 &rArr; 6月的DEMAT投資
- investment_2 &rArr; 6月份固定存款投資
- investment_3 &rArr; 6月的人壽保險投資
- investment_4 &rArr; 六月一般保險投資
- debit_amount_apr &rArr; 4月份借記的總金額
- credit_amount_apr &rArr; 4月份貸記的總金額
- debit_count_apr &rArr; 四月借記的總次數
- credit_count_apr &rArr; 四月貸記的總次數
- max_credit_amount_apr &rArr; 4月份最高金額
- debit_amount_may &rArr; 5月份借記的總金額
- credit_amount_may &rArr; 5月份貸記的總金額
- credit_count_may &rArr; 5月份貸記的總次數
- debit_count_may &rArr; 5月份借記的總次數
- max_credit_amount_may &rArr; 5月份最高金額
- debit_amount_jun &rArr; 6月借記​​的總金額
- credit_amount_jun &rArr; 6月總金額
- credit_count_jun &rArr; 6月份貸記的總次數
- debit_count_jun &rArr; 6月份借記的總次數
- max_credit_amount_jun &rArr; 6月份的最高金額
- loan_enq &rArr; 過去3個月的貸款詢問
- emi_active &rArr; 每月EMI支付給其他銀行有效貸款
- cc_cons &rArr; （目標）接下來三個月的平均信用卡花費


In [1]:
# 事前準備
import os
import numpy as np          # 資料處理分析工具
import pandas as pd         # 資料處理分析工具
from scipy import stats     # 統計函式庫
from scipy.stats import norm, skew
from collections import Counter # counter是字典，用來計數，key是要計數的item，value儲存的是個數
from sklearn.preprocessing import LabelEncoder # 用於特徵提取, 將數值資料轉為離散
from sklearn.preprocessing import MinMaxScaler

# 匯入必要的函式庫
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

# 資料視覺化
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
color = sns.color_palette() # 生成各種顏色
sns.set_style('darkgrid') # 設置主題：darkgrid、whitegrid、dark、white、ticks
# 設定展示欄位最大值
pd.set_option('display.max_row', 200) 
pd.set_option('display.max_columns', 100) 
# plt 顯示中文
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.rcParams['axes.unicode_minus'] = False
# 忽略警告
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 載入資料集
dataset = pd.read_csv('./data/Credit Card Consumption Prediction/Credit_Card_Consumption_Prediction.csv')
dataset.head(5)

,id,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,personal_loan_active,vehicle_loan_active,personal_loan_closed,vehicle_loan_closed,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active,cc_cons
0,23334,saving,M,63,305,7695.00,NaN,479.00,NaN,622.0,NaN,2.0,1.0,3.0,NaN,NaN,NaN,150000.0,NaN,NaN,NaN,NaN,NaN,118845.2,50000.0,NaN,51205.47,50259.0,3.0,1.0,50000.0,24740.22,114252.0,3.0,2.0,45000.0,20215.85,1279.37,7.0,3.0,470.0,NaN,0.00,435
1,20329,current,M,33,326,52702.00,988.72,29873.00,NaN,1087.0,2806.24,11.0,14.0,11.0,1.0,NaN,6.0,176000.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,159050.76,169812.0,20.0,9.0,70000.0,103102.85,174606.0,6.0,10.0,150000.0,122037.66,55390.00,8.0,19.0,27000.0,NaN,4963.33,1921
2,17679,current,M,44,264,10800.49,3215.52,21660.00,1607.0,1168.0,355.00,6.0,9.0,6.0,3.0,2.0,14.0,65000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51484.55,38942.0,16.0,4.0,11700.0,15283.00,23051.0,2.0,7.0,13800.0,46827.00,29297.91,6.0,19.0,20800.0,NaN,0.00,1970
3,16428,saving,M,44,233,20936.00,NaN,2094.00,NaN,11778.5,NaN,9.0,6.0,6.0,NaN,NaN,NaN,600000.0,NaN,NaN,NaN,NaN,NaN,132442.0,NaN,NaN,1981.07,2294.0,2.0,1.0,2000.0,19361.24,19529.0,1.0,1.0,19300.0,5426.55,7544.00,3.0,1.0,5300.0,NaN,1238.18,21579
4,18721,saving,F,35,324,1183.83,NaN,3656.29,NaN,945.0,NaN,2.0,7.0,9.0,NaN,NaN,NaN,100000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6462.00,10271.0,6.0,1.0,10000.0,7461.00,3194.0,2.0,3.0,3000.0,26178.05,53483.00,4.0,4.0,45000.0,NaN,0.00,1814
